<a href="https://colab.research.google.com/github/Zernach/BetterReads-DS/blob/master/How_to_Generate_Book_Recommendations_KNearestNeighbors_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#**KNN model of 10k dataset**
#_using data found on kaggle from Goodreads_
#_books.csv contains information for 10,000 books, such as ISBN, authors, title, year_
#_ratings.csv is a collection of user ratings on these books, from 1 to 5 stars_

In [2]:
# Import Statements
import numpy as pd
import pandas as pd
import pickle
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import re
import psycopg2

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


**Books dataset**

In [0]:
engine = psycopg2.connect(
    database="betterreads",
    user="betterreadsadmin",
    password="betterreadsadmin",
    host="betterreads-datascience-database.cvmplnwee5ws.us-east-1.rds.amazonaws.com",
    port='5432')

In [4]:
db_books = pd.read_sql_query("SELECT * FROM gb_data;", engine)
db_books['original_title'] = db_books['title']
db_books

,googleid,title,authors,publisher,publisheddate,description,isbn,pagecount,categories,thumbnail,smallthumbnail,lang,webreaderlink,textsnippet,isebook,averagerating,maturityrating,ratingscount,subtitle,original_title
0,PZ1cAAAACAAJ,"Black Berry, Sweet Juice",['Lawrence Hill'],HarperFlamingoCanada,2001,None,0002000202,245.0,"['Authors, Canadian (English) 20th century Bio...",None,None,en,http://play.google.com/books/reader?id=PZ1cAAA...,"The son of a black father and a white mother, ...",False,3.5,NOT_MATURE,3.0,On Being Black and White in Canada,"Black Berry, Sweet Juice"
1,sWzxAAAAMAAJ,The taste of metal,['Jack Todd'],HarperFlamingo,2001-03-29,None,WISC:89077171486,253.0,['Fiction'],http://books.google.com/books/content?id=sWzxA...,http://books.google.com/books/content?id=sWzxA...,en,http://play.google.com/books/reader?id=sWzxAAA...,He explains exactly what I need to do to get l...,False,3.0,NOT_MATURE,1.0,a deserter's story,The taste of metal
2,TbOgAAAAMAAJ,Crete on the half shell,['Byron Ayanoglu'],HarperFlamingo,2003-02-20,None,IND:30000086930678,268.0,['Cooking'],http://books.google.com/books/content?id=TbOgA...,http://books.google.com/books/content?id=TbOgA...,en,http://play.google.com/books/reader?id=TbOgAAA...,For information address: HarperCollins Publish...,False,NaN,NOT_MATURE,NaN,"a story about an island, good friends and food",Crete on the half shell
3,B5A7xgEACAAJ,Gloria,['Keith Maillard'],Harpercollins Canada,1999,Light in the Company of Women glittered with t...,0006481752,665.0,['Fiction'],http://books.google.com/books/content?id=B5A7x...,http://books.google.com/books/content?id=B5A7x...,en,http://play.google.com/books/reader?id=B5A7xgE...,None,False,4.5,NOT_MATURE,2.0,None,Gloria
4,7Pd_QgAACAAJ,Honey and Ashes,['Janice Kulyk Keefer'],Harpercollins Canada,1998,"Janice Kulyk Keefer was born in Canada, a coun...",0002554437,338.0,['Biography & Autobiography'],http://books.google.com/books/content?id=7Pd_Q...,http://books.google.com/books/content?id=7Pd_Q...,en,http://play.google.com/books/reader?id=7Pd_QgA...,"&quot;As the child of immigrants, confronted b...",False,4.0,NOT_MATURE,1.0,A Story of Family,Honey and Ashes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183153,V1duDmCqlPEC,The Life of Helen Alone,['Karen Lawrence'],None,1987,"Helen trades her safe, single life for the per...",0345330420,293.0,['Fiction'],http://books.google.com/books/content?id=V1duD...,http://books.google.com/books/content?id=V1duD...,en,http://play.google.com/books/reader?id=V1duDmC...,"Helen trades her safe, single life for the per...",False,NaN,NOT_MATURE,NaN,None,The Life of Helen Alone
183154,zi1Es2L5MvUC,Normandie Triangle,['Justin Scott'],None,1982-11,When a Nazi saboteur engineers the sinking of ...,0345306406,576.0,['Fiction'],http://books.google.com/books/content?id=zi1Es...,http://books.google.com/books/content?id=zi1Es...,en,http://play.google.com/books/reader?id=zi1Es2L...,When a Nazi saboteur engineers the sinking of ...,False,NaN,NOT_MATURE,NaN,None,Normandie Triangle
183155,y7k6CQAAQBAJ,Missing Time,['Budd hopkins'],Mantesh,1988-03-12,The arrival of extraterrestrial visitors is on...,9780345353351,260.0,"['Body, Mind & Spirit']",http://books.google.com/books/content?id=y7k6C...,http://books.google.com/books/content?id=y7k6C...,en,http://play.google.com/books/reader?id=y7k6CQA...,But Hopkins could not have told the stories of...,False,4.5,NOT_MATURE,2.0,,Missing Time
183156,_0skthx0BLAC,Perdido Street Station,['China Miéville'],Bas-Lag,2001,"In squallid and brutal New Crobuzon, a Garuda ...",0345443020,710.0,['Fiction'],http://books.google.com/books/content?id=_0skt...,http://books.google.com/books/content?id=_0skt...,en,http://play.google.com/books/reader?id=_0skthx...,"In squallid and brutal New Crobuzon, a Garuda ...",False,4.5,NOT_MATURE,17.0,None,Perdido Street Station


In [0]:
books = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv')

In [6]:
print(books.dtypes)
print(db_books.dtypes)

book_id                        int64
goodreads_book_id              int64
best_book_id                   int64
work_id                        int64
books_count                    int64
isbn                          object
isbn13                       float64
authors                       object
original_publication_year    float64
original_title                object
title                         object
language_code                 object
average_rating               float64
ratings_count                  int64
work_ratings_count             int64
work_text_reviews_count        int64
ratings_1                      int64
ratings_2                      int64
ratings_3                      int64
ratings_4                      int64
ratings_5                      int64
image_url                     object
small_image_url               object
dtype: object
googleid           object
title              object
authors            object
publisher          object
publisheddate      object
descr

In [7]:
merged_books = books.join(db_books, lsuffix='_google', rsuffix='_old')
merged_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn_google,isbn13,authors_google,original_publication_year,original_title_google,title_google,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,googleid,title_old,authors_old,publisher,publisheddate,description,isbn_old,pagecount,categories,thumbnail,smallthumbnail,lang,webreaderlink,textsnippet,isebook,averagerating,maturityrating,ratingscount,subtitle,original_title_old
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,PZ1cAAAACAAJ,"Black Berry, Sweet Juice",['Lawrence Hill'],HarperFlamingoCanada,2001,None,0002000202,245.0,"['Authors, Canadian (English) 20th century Bio...",None,None,en,http://play.google.com/books/reader?id=PZ1cAAA...,"The son of a black father and a white mother, ...",False,3.5,NOT_MATURE,3.0,On Being Black and White in Canada,"Black Berry, Sweet Juice"
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,sWzxAAAAMAAJ,The taste of metal,['Jack Todd'],HarperFlamingo,2001-03-29,None,WISC:89077171486,253.0,['Fiction'],http://books.google.com/books/content?id=sWzxA...,http://books.google.com/books/content?id=sWzxA...,en,http://play.google.com/books/reader?id=sWzxAAA...,He explains exactly what I need to do to get l...,False,3.0,NOT_MATURE,1.0,a deserter's story,The taste of metal
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,TbOgAAAAMAAJ,Crete on the half shell,['Byron Ayanoglu'],HarperFlamingo,2003-02-20,None,IND:30000086930678,268.0,['Cooking'],http://books.google.com/books/content?id=TbOgA...,http://books.google.com/books/content?id=TbOgA...,en,http://play.google.com/books/reader?id=TbOgAAA...,For information address: HarperCollins Publish...,False,NaN,NOT_MATURE,NaN,"a story about an island, good friends and food",Crete on the half shell
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,B5A7xgEACAAJ,Gloria,['Keith Maillard'],Harpercollins Canada,1999,Light in the Company of Women glittered with t...,0006481752,665.0,['Fiction'],http://books.google.com/books/content?id=B5A7x...,http://books.google.com/books/content?id=B5A7x...,en,http://play.google.com/books/reader?id=B5A7xgE...,None,False,4.5,NOT_MATURE,2.0,None,Gloria
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,7Pd_QgAACAAJ,Honey and Ashes,['Janice Kulyk Keefer'],Harpercollins Canada,1998,"Janice Kulyk Keefer was born in Canada, a coun...",0002554437,338.0,['Biography & Autobiography'],http://books.google.com/books/content?id=7Pd_Q...,http://books.google.com/books/content?id=7Pd_Q...,en,http://play.google.com/books/reader?id=7Pd_QgA...,"&quot;As the child of immigrants, confronted b...",False,4.0,NOT_MATURE,1.0,A Story of Family,Honey and Ashes


In [8]:
print(books.shape)
books.head()

(10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [9]:
books['title'].isnull().sum()

0

**Ratings dataset**

In [0]:
ratings = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv')

In [11]:
print(ratings.shape)
ratings.head()

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


**Trim down the data**

_In order to make a user rating matrix we will only need bood_id and title._



In [12]:
cols = ['book_id', 'title']

books = books[cols]
books.head()

,book_id,title
0,1,"The Hunger Games (The Hunger Games, #1)"
1,2,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,"Twilight (Twilight, #1)"
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


**Clean up book titles**

_Book titles are messy, special characters, empty spaces, brackets clutter up the titles_

In [0]:
def clean_book_titles(title):
  title = re.sub(r'\([^)]*\)', '', title) # handles brackets
  title = re.sub(' + ', ' ', title) #compresses multi spaces into a single space
  title = title.strip() # handles special characters
  return title

In [14]:
books['title'] = books['title'].apply(clean_book_titles)

books.head()

,book_id,title
0,1,The Hunger Games
1,2,Harry Potter and the Sorcerer's Stone
2,3,Twilight
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


**Create feature matrix**

_Combine datasets to get a new dataset of user ratings for each book_

In [15]:
books_ratings = pd.merge(ratings, books, on='book_id')

print(books_ratings.shape)

books_ratings.head()


(5976479, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


**Remove rows with same user_id and book title**



In [16]:
user_ratings = books_ratings.drop_duplicates(['user_id', 'title'])

print(user_ratings.shape)
user_ratings.head()

(5972713, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


**Pivot table to create user_ratings matrix**

_Each column is a user and each row is a book. The entries in the martix are the user's rating for that book._

In [17]:
user_matrix = user_ratings.pivot(index='title', columns='user_id', values='rating').fillna(0)

user_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,53385,53386,53387,53388,53389,53390,53391,53392,53393,53394,53395,53396,53397,53398,53399,53400,53401,53402,53403,53404,53405,53406,53407,53408,53409,53410,53411,53412,53413,53414,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#GIRLBOSS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
'Tis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 Places to See Before You Die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
user_matrix.shape

(9800, 53424)

**Compress the matrix since it is extremely sparse**

_Whole lotta zeros_

_

In [0]:
compressed = csr_matrix(user_matrix.values)

In [20]:
# build and train knn

# unsupervised learning
# using cosine to measure space/distance

knn = NearestNeighbors(algorithm='brute', metric='cosine')

knn.fit(compressed)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [21]:
%%time

def get_recommendations(book_title, matrix=user_matrix, model=knn, topn=5):
    book_index = list(matrix.index).index(book_title)
    distances, indices = model.kneighbors(matrix.iloc[book_index,:].values.reshape(1,-1), n_neighbors=topn+1)
    print('Recommendations for {}:'.format(matrix.index[book_index]))
    for i in range(1, len(distances.flatten())):
        print('{}. {}, distance = {}'.format(i, matrix.index[indices.flatten()[i]], "%.3f"%distances.flatten()[i]))
    print()

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [0]:
pickle.dump(knn, open('knn_model.pkl','wb'))


In [23]:
get_recommendations("Harry Potter and the Sorcerer's Stone")
get_recommendations("Pride and Prejudice")
get_recommendations("Matilda")

Recommendations for Harry Potter and the Sorcerer's Stone:
1. Harry Potter and the Prisoner of Azkaban, distance = 0.320
2. Harry Potter and the Chamber of Secrets, distance = 0.327
3. Harry Potter and the Goblet of Fire, distance = 0.331
4. Harry Potter and the Order of the Phoenix, distance = 0.342
5. Harry Potter and the Half-Blood Prince, distance = 0.348

Recommendations for Pride and Prejudice:
1. Jane Eyre, distance = 0.421
2. Sense and Sensibility, distance = 0.434
3. Little Women, distance = 0.519
4. To Kill a Mockingbird, distance = 0.520
5. Emma, distance = 0.533

Recommendations for Matilda:
1. The Witches, distance = 0.501
2. The BFG, distance = 0.525
3. James and the Giant Peach, distance = 0.540
4. Charlie and the Chocolate Factory, distance = 0.543
5. The Twits, distance = 0.689



# Testing Recommendations on Lists from Group Members

In [0]:
Ryan = ['Steve Jobs', 'How to Create A Mind', 'Rebooting AI', 'Marriageology',
        'Scale', 'The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change', 'How to Win Friends and Influence People',
        "The Tech Entrepreneur's Survival Guide", 'Measure What Matters', 'The Purpose Driven Life']

Jorge = ['Animal Farm', '1984', 'A Mind for Numbers', 'Biology of Desire', 'The Red Queen',
         'The Possibility of an Island', 'Mortality', 'Fahrenheit 451', 'Drive', 'The Antichrist']

Carl = ['Grant Cardone']

Shruti = ['Wonder', 'Triggered']

Michael = ["Ego is the Enemy", "Stillness is Key", "Everything is Fucked", 
           "Can't Hurt Me", "Way of the Warrior Kid", "What Made Maddy Run", 
           "Bed of Procrustes", "Deep Work", "Digital Minimalism", "So Good They Can't Ignore You"]

Jose = ['The 4-Hour Workweek', 'The 4-Hour Body: An Uncommon Guide to Rapid Fat-Loss, Incredible Sex, and Becoming Superhuman',
        'The Art of Learning', 'Delivered From Distraction',
        'Awaken The Giant Within', 'Waking Up: A Guide To Spirituality Without Religion',
        "So Good They Can't Ignore You", 'Antifragile', 'Outliers', 'Statistics Done Wrong']

CC = ['The Dip', 'Tribes', 'The Subtle Art of Not Giving a F*ck', 'The Alchemist',
      "Miss Peregrine’s Home for Peculiar Children", 'Hollow City', 'The 5 Love Languages: The Secret to Love that Lasts', 
      'Millennial Money', 'Just My Type', 'Graphic Design: The New Basics']

Patrick = ['Do Androids Dream of Electric Sheep? (Blade Runner, #1)',
           "The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1)",
           "Something Wicked This Way Comes (Green Town, #2)",
           "Pride and Prejudice and Zombies (Pride and Prejudice and Zombies, #1)",
           "The Curious Incident of the Dog in the Night-Time",
           "I Was Told There'd Be Cake", 'To Kill a Mockingbird', 'The Unbearable Lightness of Being',
           'The Hollow Chocolate Bunnies of the Apocalypse', 'Eats', 'Shoots & Leaves: The Zero Tolerance Approach to Punctuation']

Aaron = ["The Daily Stoic", 'Meditations', 'The Enchiridion', 'The Obstacle Is the Way: The Timeless Art of Turning Trials into Triumph',
         'Ego Is the Enemy', 'How to Think Like a Roman Emperor', 'Antifragile', 'A Man In Full',
         'Stillness Is The Key', 'The Infinite game']

Kim = ['Vanish', 'Where The Crawdads Sing', 'The Most They Ever Had', 'Crime Scene',
       'The Gifted Boss', 'The Book of Pride', 'Keto Diet', 'Business Boutique',
       'Leadership Challenge, 6th Edition', 'The Flight Girls']

Karis = ['Extremely Loud and Incredibly Close',
'The Ocean at the End of the Lane', 
'The Book Thief', 
'The Unbearable Lightness of Being', 
'The Help', 
'Talking to Strangers', 
'Girl, Wash Your Face', 
'Eleanor Oliphant Is Completely Fine', 
'Fight Club', 
'The Ride of a Lifetime', 
'Bossypants', 
'World War Z', 
'Darkly Dreaming Dexter', 
'The Wind-Up Bird Chronicle', 
'Smoke and Mirrors', 
'Z: A Novel of Zelda Fitzgerald', 
'Eat, Pray, Love', 
'The Reader on the 6.27', 
'Fahrenheit 451', 
'This Side of Paradise', 
'If You Feel Too Much']

Allison = ['1984', 
'Switching Well', 
'Beaches', 
'Animal Farm', 
'Speak', 
'Not a Fan', 
'Heads in Beds', 
'My sisters Keeper', 
'Lock and Key']

Thom = ['The Art Of War', 'On the Origin of the Species', 'The Comedy of Errors', 
        'A Brief History of Time', 'The Second Chronicles of Thomas Covenant: The Unbeliever',
        'What If Venom Possessed Deadpool', 'Gantz', 'The Very Hungry Caterpillar',
        'The Book of Ninja', 'Beauty Behind the Madness']

In [0]:
def recs(book_list):
    for book in book_list:
        try:
            get_recommendations(book)
        except Exception:
            print(book + ' is not in the list.\n')

In [26]:
recs(Ryan)

Recommendations for Steve Jobs:
1. Outliers: The Story of Success, distance = 0.817
2. The Tipping Point: How Little Things Can Make a Big Difference, distance = 0.825
3. The Snowball: Warren Buffett and the Business of Life, distance = 0.826
4. Einstein: His Life and Universe, distance = 0.833
5. Freakonomics: A Rogue Economist Explores the Hidden Side of Everything, distance = 0.834

How to Create A Mind is not in the list.

Rebooting AI is not in the list.

Marriageology is not in the list.

Scale is not in the list.

Recommendations for The 7 Habits of Highly Effective People: Powerful Lessons in Personal Change:
1. How to Win Friends and Influence People, distance = 0.626
2. Who Moved My Cheese?, distance = 0.641
3. The One Minute Manager, distance = 0.642
4. Think and Grow Rich: The Landmark Bestseller - Now Revised and Updated for the 21st Century, distance = 0.681
5. The Power of Positive Thinking, distance = 0.712

Recommendations for How to Win Friends and Influence People:
1

In [27]:
recs(Jorge)

Recommendations for Animal Farm:
1. 1984, distance = 0.376
2. Lord of the Flies, distance = 0.467
3. The Catcher in the Rye, distance = 0.489
4. The Great Gatsby, distance = 0.507
5. To Kill a Mockingbird, distance = 0.510

Recommendations for 1984:
1. Animal Farm, distance = 0.376
2. Brave New World, distance = 0.452
3. The Catcher in the Rye, distance = 0.499
4. Lord of the Flies, distance = 0.501
5. The Great Gatsby, distance = 0.511

A Mind for Numbers is not in the list.

Biology of Desire is not in the list.

Recommendations for The Red Queen:
1. The Lady of the Rivers, distance = 0.445
2. The Kingmaker's Daughter, distance = 0.526
3. The White Queen, distance = 0.527
4. The Other Queen, distance = 0.564
5. The White Princess, distance = 0.574

The Possibility of an Island is not in the list.

Recommendations for Mortality:
1. Hitch-22: A Memoir, distance = 0.818
2. god is Not Great: How Religion Poisons Everything, distance = 0.873
3. Free Will, distance = 0.881
4. The Portable 

In [28]:
recs(Carl)

Grant Cardone is not in the list.



In [29]:
recs(Shruti)

Recommendations for Wonder:
1. The Fault in Our Stars, distance = 0.732
2. The One and Only Ivan, distance = 0.751
3. Me Before You, distance = 0.763
4. Eleanor & Park, distance = 0.764
5. The Girl on the Train, distance = 0.782

Triggered is not in the list.



In [30]:
recs(Michael)

Ego is the Enemy is not in the list.

Stillness is Key is not in the list.

Everything is Fucked is not in the list.

Can't Hurt Me is not in the list.

Way of the Warrior Kid is not in the list.

What Made Maddy Run is not in the list.

Bed of Procrustes is not in the list.

Deep Work is not in the list.

Digital Minimalism is not in the list.

So Good They Can't Ignore You is not in the list.



In [31]:
recs(Jose)

Recommendations for The 4-Hour Workweek:
1. Crush It!: Why Now Is the Time to Cash In on Your Passion, distance = 0.754
2. The 4-Hour Body: An Uncommon Guide to Rapid Fat-Loss, Incredible Sex, and Becoming Superhuman, distance = 0.758
3. Think and Grow Rich: The Landmark Bestseller - Now Revised and Updated for the 21st Century, distance = 0.774
4. Getting Things Done: The Art of Stress-Free Productivity, distance = 0.776
5. How to Win Friends and Influence People, distance = 0.792

Recommendations for The 4-Hour Body: An Uncommon Guide to Rapid Fat-Loss, Incredible Sex, and Becoming Superhuman:
1. The 4-Hour Workweek, distance = 0.758
2. The Obstacle Is the Way: The Timeless Art of Turning Trials into Triumph, distance = 0.847
3. I Will Teach You To Be Rich, distance = 0.879
4. Never Eat Alone: And Other Secrets to Success, One Relationship at a Time, distance = 0.902
5. How to Win Friends and Influence People, distance = 0.907

The Art of Learning is not in the list.

Delivered From 

In [32]:
recs(CC)

The Dip is not in the list.

Tribes is not in the list.

The Subtle Art of Not Giving a F*ck is not in the list.

Recommendations for The Alchemist:
1. The Kite Runner, distance = 0.662
2. The Da Vinci Code, distance = 0.678
3. To Kill a Mockingbird, distance = 0.688
4. Life of Pi, distance = 0.696
5. The Catcher in the Rye, distance = 0.705

Recommendations for Miss Peregrine’s Home for Peculiar Children:
1. Hollow City, distance = 0.556
2. Library of Souls, distance = 0.695
3. The Fault in Our Stars, distance = 0.732
4. Divergent, distance = 0.735
5. The Hunger Games, distance = 0.753

Recommendations for Hollow City:
1. Library of Souls, distance = 0.385
2. Miss Peregrine’s Home for Peculiar Children, distance = 0.556
3. Allegiant, distance = 0.828
4. Harry Potter and the Cursed Child - Parts One and Two, distance = 0.836
5. Insurgent, distance = 0.843

The 5 Love Languages: The Secret to Love that Lasts is not in the list.

Millennial Money is not in the list.

Just My Type is not 

In [33]:
recs(Patrick)

Do Androids Dream of Electric Sheep? (Blade Runner, #1) is not in the list.

The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy, #1) is not in the list.

Something Wicked This Way Comes (Green Town, #2) is not in the list.

Pride and Prejudice and Zombies (Pride and Prejudice and Zombies, #1) is not in the list.

Recommendations for The Curious Incident of the Dog in the Night-Time:
1. Life of Pi, distance = 0.595
2. The Kite Runner, distance = 0.611
3. The Lovely Bones, distance = 0.639
4. To Kill a Mockingbird, distance = 0.644
5. The Time Traveler's Wife, distance = 0.656

Recommendations for I Was Told There'd Be Cake:
1. When You Are Engulfed in Flames, distance = 0.886
2. Dress Your Family in Corduroy and Denim, distance = 0.890
3. Is Everyone Hanging Out Without Me?, distance = 0.893
4. Bossypants, distance = 0.898
5. Holidays on Ice, distance = 0.901

Recommendations for To Kill a Mockingbird:
1. The Great Gatsby, distance = 0.412
2. The Catcher in the Rye, 

In [40]:
recs(Aaron)

The Daily Stoic is not in the list.

Recommendations for Meditations:
1. The Prince, distance = 0.824
2. The Obstacle Is the Way: The Timeless Art of Turning Trials into Triumph, distance = 0.834
3. The Republic, distance = 0.837
4. The Nicomachean Ethics, distance = 0.843
5. Politics, distance = 0.853

The Enchiridion is not in the list.

Recommendations for The Obstacle Is the Way: The Timeless Art of Turning Trials into Triumph:
1. Zero to One: Notes on Startups, or How to Build the Future, distance = 0.804
2. Essentialism: The Disciplined Pursuit of Less, distance = 0.828
3. Meditations, distance = 0.834
4. Mastery, distance = 0.839
5. Elon Musk: Inventing the Future, distance = 0.839

Ego Is the Enemy is not in the list.

How to Think Like a Roman Emperor is not in the list.

Antifragile is not in the list.

A Man In Full is not in the list.

Stillness Is The Key is not in the list.

The Infinite game is not in the list.



In [35]:
recs(Kim)

Recommendations for Vanish:
1. Body Double, distance = 0.465
2. The Sinner, distance = 0.472
3. The Apprentice, distance = 0.508
4. The Keepsake, distance = 0.512
5. The Mephisto Club, distance = 0.515

Where The Crawdads Sing is not in the list.

The Most They Ever Had is not in the list.

Crime Scene is not in the list.

The Gifted Boss is not in the list.

The Book of Pride is not in the list.

Keto Diet is not in the list.

Business Boutique is not in the list.

Leadership Challenge, 6th Edition is not in the list.

The Flight Girls is not in the list.



In [36]:
recs(Karis)

Recommendations for Extremely Loud and Incredibly Close:
1. Everything Is Illuminated, distance = 0.592
2. The Curious Incident of the Dog in the Night-Time, distance = 0.749
3. The History of Love, distance = 0.758
4. Life of Pi, distance = 0.770
5. Middlesex, distance = 0.771

Recommendations for The Ocean at the End of the Lane:
1. American Gods, distance = 0.706
2. Neverwhere, distance = 0.722
3. The Graveyard Book, distance = 0.731
4. Anansi Boys, distance = 0.745
5. Stardust, distance = 0.769

Recommendations for The Book Thief:
1. The Help, distance = 0.609
2. The Hunger Games, distance = 0.609
3. The Fault in Our Stars, distance = 0.621
4. The Kite Runner, distance = 0.637
5. Gone Girl, distance = 0.648

Recommendations for The Unbearable Lightness of Being:
1. One Hundred Years of Solitude, distance = 0.700
2. Love in the Time of Cholera, distance = 0.702
3. The Book of Laughter and Forgetting, distance = 0.722
4. Lolita, distance = 0.723
5. The Stranger, distance = 0.733

Rec

In [37]:
recs(Allison)

Recommendations for 1984:
1. Animal Farm, distance = 0.376
2. Brave New World, distance = 0.452
3. The Catcher in the Rye, distance = 0.499
4. Lord of the Flies, distance = 0.501
5. The Great Gatsby, distance = 0.511

Switching Well is not in the list.

Recommendations for Beaches:
1. The First Wives Club, distance = 0.597
2. Terms of Endearment, distance = 0.640
3. Dances with Wolves, distance = 0.684
4. Driving Miss Daisy, distance = 0.695
5. Dead Poets Society, distance = 0.705

Recommendations for Animal Farm:
1. 1984, distance = 0.376
2. Lord of the Flies, distance = 0.467
3. The Catcher in the Rye, distance = 0.489
4. The Great Gatsby, distance = 0.507
5. To Kill a Mockingbird, distance = 0.510

Recommendations for Speak:
1. Thirteen Reasons Why, distance = 0.726
2. Wintergirls, distance = 0.733
3. Holes, distance = 0.735
4. Stargirl, distance = 0.746
5. The Perks of Being a Wallflower, distance = 0.754

Not a Fan is not in the list.

Heads in Beds is not in the list.

My sisters

In [38]:
recs(Thom)

The Art Of War is not in the list.

On the Origin of the Species is not in the list.

Recommendations for The Comedy of Errors:
1. Antony and Cleopatra, distance = 0.757
2. Henry V, distance = 0.767
3. Richard III, distance = 0.775
4. Titus Andronicus, distance = 0.777
5. The Winter's Tale, distance = 0.780

Recommendations for A Brief History of Time:
1. The Elegant Universe: Superstrings, Hidden Dimensions, and the Quest for the Ultimate Theory, distance = 0.714
2. Cosmos, distance = 0.716
3. The Universe in a Nutshell, distance = 0.754
4. The Grand Design, distance = 0.772
5. The Selfish Gene, distance = 0.778

The Second Chronicles of Thomas Covenant: The Unbeliever is not in the list.

What If Venom Possessed Deadpool is not in the list.

Gantz is not in the list.

The Very Hungry Caterpillar is not in the list.

The Book of Ninja is not in the list.

Beauty Behind the Madness is not in the list.

